# Understanding Memory in LLMs

In the previous Notebooks, we successfully explored how OpenAI models can enhance the results from Azure AI Search queries. 

However, we have yet to discover how to engage in a conversation with the LLM. With [Bing Chat](http://chat.bing.com/), for example, this is possible, as it can understand and reference the previous responses.

There is a common misconception that LLMs (Large Language Models) have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

In this Notebook, our goal is to illustrate how we can effectively "endow the LLM with memory" by employing prompts and context.

In [36]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List
import openai
from azure.identity import ManagedIdentityCredential

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever, get_answer
from common.prompts import DOCSEARCH_PROMPT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [4]:
QUESTION = "Tell me some use cases for reinforcement learning"
FOLLOW_UP_QUESTION = "What was my prior question?"

In [7]:
# Step 1: Specify the client ID of the User Managed Identity
user_managed_identity_client_id = "d30cba06-04c1-4065-a91d-8b7ce3b07b78"  # Replace with your User Managed Identity client ID

# Step 2: Fetch the access token using ManagedIdentityCredential and the client ID of the user-managed identity
credential = ManagedIdentityCredential(client_id=user_managed_identity_client_id)
token = credential.get_token("https://cognitiveservices.azure.com/.default")

# Step 3: Set the access token in the OpenAI API
openai.api_key = token.token
openai.api_type = "azure"
openai.api_base = "https://azuremlopenai.openai.azure.com/"  # Replace with your OpenAI resource's base URL
openai.api_version = "2023-06-01-preview"  # Use the correct API version

In [9]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(openai_api_key=token.token,azure_endpoint=openai.api_base,openai_api_version=openai.api_version,deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS,)

In [10]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}")
])

In [11]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

Reinforcement Learning (RL) is a type of machine learning where an agent learns to make decisions by performing certain actions and receiving rewards or penalties in return. This trial-and-error approach allows the agent to learn optimal behaviors over time. Here are some compelling use cases for reinforcement learning:

### 1. **Robotics**
- **Autonomous Navigation**: RL can be used to train robots to navigate through complex environments without human intervention.
- **Manipulation Tasks**: Robots can learn to handle objects, assemble parts, or perform other intricate tasks in manufacturing or service industries.

### 2. **Gaming**
- **Game Playing**: RL has been used to create agents that can play and excel at complex games like Go, Chess, and video games. Notable examples include AlphaGo and OpenAI's Dota 2 agent.
- **Game Design**: RL can also be used to design game levels that adapt to the player's skill level, providing a more engaging experience.

### 3. **Finance**
- **Algorithmic Trading**: RL algorithms can be trained to make trading decisions based on historical data and market conditions, aiming to maximize returns.
- **Portfolio Management**: RL can help in dynamically adjusting the composition of a portfolio to optimize for risk and return.

### 4. **Healthcare**
- **Personalized Treatment**: RL can be used to develop personalized treatment plans for patients, adjusting dosages and treatment types based on individual responses.
- **Resource Management**: Hospitals can use RL to optimize the allocation of resources like beds, staff, and equipment.

### 5. **Autonomous Vehicles**
- **Self-Driving Cars**: RL is crucial in training autonomous vehicles to make real-time decisions for navigation, obstacle avoidance, and adherence to traffic laws.
- **Fleet Management**: Companies can use RL to optimize routes, schedules, and maintenance for fleets of autonomous vehicles.

### 6. **Natural Language Processing (NLP)**
- **Dialogue Systems**: RL can improve conversational agents or chatbots by optimizing their responses based on user interactions.
- **Machine Translation**: RL can be used to enhance the quality of machine translation systems by rewarding more accurate translations.

### 7. **Energy Management**
- **Smart Grids**: RL can optimize the distribution of electricity in smart grids, balancing supply and demand more efficiently.
- **Building Energy Management**: RL can be used to control heating, ventilation, and air conditioning (HVAC) systems to minimize energy consumption while maintaining comfort.

### 8. **Marketing**
- **Ad Placement**: RL can optimize the placement and timing of ads to maximize engagement and conversion rates.
- **Customer Segmentation**: RL can help in dynamically segmenting customers based on their behavior, allowing for more personalized marketing strategies.

### 9. **Supply Chain Optimization**
- **Inventory Management**: RL can optimize inventory levels, reducing holding costs and stockouts.
- **Logistics and Routing**: RL can improve the efficiency of logistics networks by optimizing routes and schedules for deliveries.

### 10. **Telecommunications**
- **Network Optimization**: RL can be used to manage and optimize network traffic, reducing latency and improving service quality.
- **Dynamic Spectrum Allocation**: RL can help in dynamically allocating spectrum resources to different users or services, optimizing the overall network performance.

### 11. **Education**
- **Personalized Learning**: RL can be used to create adaptive learning systems that tailor educational content to the needs and progress of individual students.
- **Tutoring Systems**: RL can optimize the interaction between students and tutoring systems, providing more effective feedback and guidance.

### 12. **Manufacturing**
- **Predictive Maintenance**: RL can predict equipment failures and optimize maintenance schedules, reducing downtime and costs.
- **Process Optimization**: RL can optimize manufacturing processes to improve efficiency and product quality.

These are just a few examples, and the potential applications of reinforcement learning continue to grow as the technology advances.

In [12]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

I'm sorry, but I don't have access to previous interactions. How can I assist you today?

As you can see, it doesn't remember what it just responded, sometimes it responds based only on the system prompt, or just randomly. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [13]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [14]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [15]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

Your prior question was: "Tell me some use cases for reinforcement learning."

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

From Langchain website:
    
A memory system needs to support two basic actions: reading and writing. Recall that every chain defines some core execution logic that expects certain inputs. Some of these inputs come directly from the user, but some of these inputs can come from memory. A chain will interact with its memory system twice in a given run.

    AFTER receiving the initial user inputs but BEFORE executing the core logic, a chain will READ from its memory system and augment the user inputs.
    AFTER executing the core logic but BEFORE returning the answer, a chain will WRITE the inputs and outputs of the current run to memory, so that they can be referred to in future runs.
    
So this process adds delays to the response, but it is a necessary delay :)

![image](./images/memory_diagram.png)

In [16]:
index1_name = "srch-index-files-yk"
index2_name = "srch-index-csv-yk"
index3_name = "srch-index-books-yk"
indexes = [index1_name, index2_name, index3_name]

In [19]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=10, reranker_threshold=1)

If you check closely in prompts.py, there is an optional variable in the `DOCSEARCH_PROMPT` called `history`. Now it is the time to use it. It is basically a place holder were we will inject the conversation in the prompt so the LLM is aware of it before it answers.

**Now let's add memory to it:**

In [40]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [41]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [42]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123"}}

Notice below, that we are adding a `history` variable in the call. This variable will hold the chat historywithin the prompt.

In [43]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was: "Tell me some use cases for reinforcement learning."

In [22]:
# Remembers
printmd(chain_with_history.invoke({"question": "Thank you! Good bye"},config=config))

You're welcome! If you have any more questions in the future, feel free to ask. Goodbye!

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wish to provide recommendations in the future. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory

In [44]:
credential = ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")

In [45]:
# Create the function to retrieve the conversation

def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        credential=credential,
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [46]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
) | output_parser

In [47]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

In [48]:
config

{'configurable': {'session_id': 'session83', 'user_id': 'user355'}}

In [50]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))
#Cosmosdb role assignment by power shell is needed
# $resourceGroupName="ai-bootcamp"
# $accountName="cosmosdb-account-jed5nzg3k2jp6"
# $readOnlyRoleDefinitionId="00000000-0000-0000-0000-000000000002"
# $principalId="ce7ea55d-acf9-4107-a891-ca04f4d417b3"
# az cosmosdb sql role assignment create --account-name $accountName --resource-group $resourceGroupName --scope "/" --principal-id $principalId --role-definition-id $readOnlyRoleDefinitionId



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [28]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Your prior question was: "Tell me some use cases for reinforcement learning."

In [29]:
# Remembers
printmd(chain_with_history.invoke(
    {"question": "Can you tell me a one line summary of our conversation?"},
    config=config))

We discussed various use cases of reinforcement learning, including applications in games, robotics, healthcare, and adaptive control systems.

In [30]:
try:
    printmd(chain_with_history.invoke(
    {"question": "Thank you very much!"},
    config=config))
except Exception as e:
    print(e)

You're welcome! If you have any more questions, feel free to ask. Have a great day!

In [31]:
printmd(chain_with_history.invoke(
    {"question": "I do have one more question, why did you give me a one line summary?"},
    config=config))

I provided a one-line summary of our conversation because you requested it in your previous message. If you need more detailed information or have another question, feel free to ask!

In [32]:
printmd(chain_with_history.invoke(
    {"question": "why not 2?"},
    config=config))

I apologize for any confusion. If you would like a more detailed summary, here it is in two lines:

We discussed various use cases of reinforcement learning, including applications in games like backgammon and chess, robotics such as juggling robots and mobile robots, healthcare through epidemic modeling, and adaptive control systems. These examples illustrate the versatility of reinforcement learning in solving complex problems across various fields.

Feel free to ask if you need more information or have any other questions!

#### Let's check our Azure CosmosDB to see the whole conversation


![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, regardless of the input. This doesn't seem efficient, but regardless, we are very close to finish our first RAG-talk to your data Bot.


## <u>Important Note</u>:<br>
As we proceed, while all the code will remain compatible with GPT-3.5 models (1106 or newer), we highly recommend transitioning to GPT-4. Here's why:

**GPT-3.5-Turbo** can be likened to a 7-year-old child. You can provide it with concise instructions, but it struggles sometimes to follow them accurately (not too reliable). Additionally, its limited "memory" (token context) can make sustained conversations challenging. Its response are also simple not deep.

**GPT-4** exhibits the capabilities of a 10-12-year-old child. It possesses enhanced reasoning skills, consistently adheres to instructions and its answers are beter. It has extended memory retention (larger context size) for instructions, and it excels at following them. Its responses are deep and thorough.


# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

In the next notebook 6, we are going to build our first RAG bot. In order to do this we will introduce the concept of Agents.